In [1]:
from sklearn import svm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
import warnings
from collections import Counter
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
from keras import Sequential, Model
from keras.layers import Dense
import keras
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils

In [2]:
df= pd.read_csv('bankloan.csv')
df= df.dropna()
df.isna().any()
df=df.drop('Loan_ID', axis=1)
df['LoanAmount']= (df['LoanAmount']*1000).astype(int)
Counter(df['Loan_Status'])

Counter({'N': 148, 'Y': 332})

In [3]:
pre_y= df['Loan_Status']
pre_x= df.drop('Loan_Status', axis=1)
dm_x= pd.get_dummies(pre_x)
dm_y=pre_y.map(dict(Y=1, N=0))
pre_y.shape

(480,)

In [58]:
dm_x.columns

Index(['Dependents', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Gender_Female', 'Gender_Male',
       'Married_No', 'Married_Yes', 'Education_Graduate',
       'Education_Not Graduate', 'Self_Employed_No', 'Self_Employed_Yes',
       'Property_Area_Rural', 'Property_Area_Semiurban',
       'Property_Area_Urban'],
      dtype='object')

In [61]:
import joblib
joblib.dump(dm_x.columns,'dm_x.pkl')

['dm_x.pkl']

In [59]:
df.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [60]:
cat_columns= ['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area']
pre_x_processed= pd.get_dummies(pre_x, columns=cat_columns) 
pre_x_processed.columns==dm_x.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True])

In [4]:
smote = SMOTE(sampling_strategy='minority')
X1, y= smote.fit_sample(dm_x, dm_y)
sc= MinMaxScaler()
X= sc.fit_transform(X1)

In [5]:
import joblib
joblib.dump(sc,'scaler.pkl')
#sc = joblib.load('scaler.pkl')

['scaler.pkl']

In [6]:
x_train, x_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=42, shuffle= True)

In [7]:
classifier = Sequential()
classifier.add(Dense(200, activation= 'relu',
                     kernel_initializer= 'random_normal',
                     input_dim=x_test.shape[1]))
classifier.add(Dense(400, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classifier.fit(x_train, y_train, batch_size=5, epochs=50, verbose=0)
eval_model=classifier.evaluate(x_train,y_train)
eval_model

17/17 [==============================] - 0s 646us/step - loss: 0.2393 - accuracy: 0.9077


[0.23932801187038422, 0.9077212810516357]

In [8]:
y_pred= classifier.predict(x_test)
y_pred= (y_pred>0.5)

In [9]:
cm= confusion_matrix(y_test, y_pred)
cm

array([[51, 14],
       [ 5, 63]])

In [26]:
def unpack(model, training_config, weights):
    restored_model = deserialize(model)
    if training_config is not None:
        restored_model.compile(
            **saving_utils.compile_args_from_training_config(
                training_config
            )
        )
    restored_model.set_weights(weights)
    return restored_model

# Hotfix function
def make_keras_picklable():

    def __reduce__(self):
        model_metadata = saving_utils.model_metadata(self)
        training_config = model_metadata.get("training_config", None)
        model = serialize(self)
        weights = self.get_weights()
        return (unpack, (model, training_config, weights))

    cls = Model
    cls.__reduce__ = __reduce__
    

# Run the function
make_keras_picklable()


In [27]:
import joblib
joblib.dump(classifier, 'loan_model.pkl')

['loan_model.pkl']

In [33]:
ml=joblib.load('loan_model.pkl')

In [29]:
y_pred=ml.predict(x_test)
y_pred= (y_pred> 0.5)
val= pd.DataFrame(y_pred, columns=['Status'])
val= val.replace({True:'Approved', False: 'Rejected'})
val

,Status
0,Approved
1,Rejected
2,Approved
3,Approved
4,Approved
...,...
128,Rejected
129,Rejected
130,Rejected
131,Approved


In [10]:
import types
import tempfile
import keras.models

def make_keras_picklable():
    def __getstate__(self):
        model_str = ""
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            keras.models.save_model(self, fd.name, overwrite=True)
            model_str = fd.read()
        d = { 'model_str': model_str }
        return d

    def __setstate__(self, state):
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            fd.write(state['model_str'])
            fd.flush()
            model = keras.models.load_model(fd.name)
        self.__dict__ = model.__dict__


    cls = keras.models.Model
    cls.__getstate__ = __getstate__
    cls.__setstate__ = __setstate__

make_keras_picklable()

In [23]:
classifier.save("Mlmodel.h5")

In [24]:
mlh5=keras.models.load_model("Mlmodel.h5")

In [36]:
x_test[0]
iter(x_test[0])
dict(zip(iter(x_test[0]),iter(x_test[0])))

{0.0: 0.0,
 0.056895485466914045: 0.056895485466914045,
 0.0689481928066909: 0.0689481928066909,
 0.2047377326565144: 0.2047377326565144,
 0.7297297297297298: 0.7297297297297298,
 1.0: 1.0}

In [52]:
[f"{i}" for i in 'abcdefghijklmnopqrstuvwxyz'][0:17]

['a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q']

In [41]:
len(dict(zip(rand_letters, x_test[0])))

15

In [55]:
import json
json.dumps(dict(zip([i for i in 'abcdefghijklmnopqrstuvwxyz'][0:17], x_test[0])))

'{"a": 0.0, "b": 0.056895485466914045, "c": 0.0689481928066909, "d": 0.2047377326565144, "e": 0.7297297297297298, "f": 1.0, "g": 0.0, "h": 1.0, "i": 0.0, "j": 1.0, "k": 1.0, "l": 0.0, "m": 1.0, "n": 0.0, "o": 0.0, "p": 0.0, "q": 1.0}'

In [25]:
y_pred=mlh5.predict(x_test)
y_pred= (y_pred> 0.5)
val= pd.DataFrame(y_pred, columns=['Status'])
val= val.replace({True:'Approved', False: 'Rejected'})
val

,Status
0,Approved
1,Rejected
2,Approved
3,Approved
4,Approved
...,...
128,Rejected
129,Rejected
130,Rejected
131,Approved
